In [ ]:
import os
import glob
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.utils as vutils
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
from skimage.metrics import structural_similarity as compare_ssim
import numpy as np
import time

# === Model Definition ===
class DeepDehazeNet(nn.Module):
    def __init__(self, layers=8):
        super().__init__()
        def conv_block(in_c, out_c):
            return nn.Sequential(
                nn.Conv2d(in_c, out_c, 3, padding=1),
                nn.BatchNorm2d(out_c),
                nn.ReLU(inplace=True),
                nn.Dropout(0.2)
            )
        self.layers = layers
        if layers == 4:
            self.enc1 = conv_block(3, 64)
            self.pool1 = nn.MaxPool2d(2)
            self.bottleneck = conv_block(64, 128)
            self.up1 = nn.ConvTranspose2d(128, 64, 2, stride=2)
            self.dec1 = conv_block(128, 64)
            self.final = nn.Conv2d(64, 3, 1)
        elif layers == 8:
            self.enc1 = conv_block(3, 64)
            self.pool1 = nn.MaxPool2d(2)
            self.enc2 = conv_block(64, 128)
            self.pool2 = nn.MaxPool2d(2)
            self.enc3 = conv_block(128, 256)
            self.pool3 = nn.MaxPool2d(2)
            self.bottleneck = conv_block(256, 512)
            self.up1 = nn.ConvTranspose2d(512, 256, 2, stride=2)
            self.dec1 = conv_block(512, 256)
            self.up2 = nn.ConvTranspose2d(256, 128, 2, stride=2)
            self.dec2 = conv_block(256, 128)
            self.up3 = nn.ConvTranspose2d(128, 64, 2, stride=2)
            self.dec3 = conv_block(128, 64)
            self.final = nn.Conv2d(64, 3, 1)
        elif layers == 16:
            self.enc1 = conv_block(3, 64)
            self.pool1 = nn.MaxPool2d(2)
            self.enc2 = conv_block(64, 128)
            self.pool2 = nn.MaxPool2d(2)
            self.enc3 = conv_block(128, 256)
            self.pool3 = nn.MaxPool2d(2)
            self.enc4 = conv_block(256, 512)
            self.pool4 = nn.MaxPool2d(2)
            self.enc5 = conv_block(512, 1024)
            self.pool5 = nn.MaxPool2d(2)
            self.bottleneck = conv_block(1024, 2048)
            self.up1 = nn.ConvTranspose2d(2048, 1024, 2, stride=2)
            self.dec1 = conv_block(2048, 1024)
            self.up2 = nn.ConvTranspose2d(1024, 512, 2, stride=2)
            self.dec2 = conv_block(1024, 512)
            self.up3 = nn.ConvTranspose2d(512, 256, 2, stride=2)
            self.dec3 = conv_block(512, 256)
            self.up4 = nn.ConvTranspose2d(256, 128, 2, stride=2)
            self.dec4 = conv_block(256, 128)
            self.up5 = nn.ConvTranspose2d(128, 64, 2, stride=2)
            self.dec5 = conv_block(128, 64)
            self.final = nn.Conv2d(64, 3, 1)

    def forward(self, x):
        if self.layers == 4:
            e1 = self.enc1(x)
            b = self.bottleneck(self.pool1(e1))
            d1 = self.dec1(torch.cat([self.up1(b), e1], 1))
            return torch.sigmoid(self.final(d1))
        elif self.layers == 8:
            e1 = self.enc1(x)
            e2 = self.enc2(self.pool1(e1))
            e3 = self.enc3(self.pool2(e2))
            b = self.bottleneck(self.pool3(e3))
            d1 = self.dec1(torch.cat([self.up1(b), e3], 1))
            d2 = self.dec2(torch.cat([self.up2(d1), e2], 1))
            d3 = self.dec3(torch.cat([self.up3(d2), e1], 1))
            return torch.sigmoid(self.final(d3))
        elif self.layers == 16:
            e1 = self.enc1(x)
            e2 = self.enc2(self.pool1(e1))
            e3 = self.enc3(self.pool2(e2))
            e4 = self.enc4(self.pool3(e3))
            e5 = self.enc5(self.pool4(e4))
            b = self.bottleneck(self.pool5(e5))
            d1 = self.dec1(torch.cat([self.up1(b), e5], 1))
            d2 = self.dec2(torch.cat([self.up2(d1), e4], 1))
            d3 = self.dec3(torch.cat([self.up3(d2), e3], 1))
            d4 = self.dec4(torch.cat([self.up4(d3), e2], 1))
            d5 = self.dec5(torch.cat([self.up5(d4), e1], 1))
            return torch.sigmoid(self.final(d5))

# === Setup ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# === Paths ===
hazy_dir = "hazy"
clean_dir = "clear"
results_dir = "results_final"
os.makedirs(results_dir, exist_ok=True)

# === Load image paths and randomly sample 10 pairs ===
all_hazy_paths = sorted(glob.glob(os.path.join(hazy_dir, "*")))
all_clean_paths = sorted(glob.glob(os.path.join(clean_dir, "*")))

paired_paths = list(zip(all_hazy_paths, all_clean_paths))
random.shuffle(paired_paths)
selected_pairs = paired_paths[:30]

hazy_imgs = [transform(Image.open(p[0]).convert("RGB")) for p in selected_pairs]
clean_imgs = [transform(Image.open(p[1]).convert("RGB")) for p in selected_pairs]

# === Load Models ===
models = {
    "4": DeepDehazeNet(4).to(device),
    "8": DeepDehazeNet(8).to(device),
    "16": DeepDehazeNet(16).to(device),
}
models["4"].load_state_dict(torch.load("4_layers_model/best_model_4_4.pth", map_location=device))
models["8"].load_state_dict(torch.load("8_layers_model/best_model_8_8.pth", map_location=device))
models["16"].load_state_dict(torch.load("16_layers_model/best_model_16_16.pth", map_location=device))
for m in models.values():
    m.eval()

# === Inference and Evaluation ===
rows = []
psnr_dict, ssim_dict, time_dict = {"4": [], "8": [], "16": []}, {"4": [], "8": [], "16": []}, {"4": [], "8": [], "16": []}
font = ImageFont.load_default()

for i in range(30):
    hazy = hazy_imgs[i].unsqueeze(0).to(device)
    clean = clean_imgs[i].unsqueeze(0).to(device)

    row = [hazy_imgs[i], clean_imgs[i]]
    for k in ["4", "8", "16"]:
        start = time.time()
        with torch.no_grad():
            output = models[k](hazy)
        elapsed = time.time() - start
        output_img = output.squeeze().cpu()
        row.append(output_img)

        psnr_val = compare_psnr(clean.squeeze().cpu().permute(1, 2, 0).numpy(),
                                output_img.permute(1, 2, 0).numpy(), data_range=1.0)
        ssim_val = compare_ssim(clean.squeeze().cpu().permute(1, 2, 0).numpy(),
                                output_img.permute(1, 2, 0).numpy(), data_range=1.0, channel_axis=-1)

        psnr_dict[k].append(psnr_val)
        ssim_dict[k].append(ssim_val)
        time_dict[k].append(elapsed)

        # Create label image
        label_img = Image.new("RGB", (256, 256), (0, 0, 0))
        d = ImageDraw.Draw(label_img)
        d.text((10, 50), f"PSNR: {psnr_val:.2f}", fill="white", font=font)
        d.text((10, 100), f"SSIM: {ssim_val:.3f}", fill="white", font=font)
        d.text((10, 150), f"Time: {elapsed:.4f}s", fill="white", font=font)
        row.append(transforms.ToTensor()(label_img))

    rows.append(torch.stack(row))

# === Create Grid Image ===
grid = torch.cat(rows, dim=0)
grid_img = vutils.make_grid(grid, nrow=8, padding=2)
grid_img_np = (grid_img.permute(1, 2, 0).numpy() * 255).astype(np.uint8)
img = Image.fromarray(grid_img_np)
draw = ImageDraw.Draw(img)

# Add column labels
columns = ["Hazy", "GT", "4L Output", "4L Metrics", "8L Output", "8L Metrics", "16L Output", "16L Metrics"]
for idx, label in enumerate(columns):
    draw.text((idx * 256 + 10, 5), label, fill="white", font=font)

img.save(os.path.join(results_dir, "comparison_table_full.png"))
print("✅ Saved full comparison table with labels and metrics.")

# === PSNR & SSIM Line Plots ===
x = list(range(1, 31))
for metric, name, dictionary in [(psnr_dict, "PSNR", psnr_dict), (ssim_dict, "SSIM", ssim_dict)]:
    plt.figure(figsize=(10, 6))
    for k in ["4", "8", "16"]:
        plt.plot(x, dictionary[k], label=f"{k} Layers")
    plt.xlabel("Image Index")
    plt.ylabel(name)
    plt.title(f"{name} vs Image Index")
    plt.legend()
    plt.grid()
    plt.savefig(os.path.join(results_dir, f"{name.lower()}_vs_index.png"))
    plt.close()
    print(f"✅ Saved {name} vs index plot.")

# === Inference Time Bar Graph (Per Image) ===
bar_width = 0.25
x_indexes = np.arange(30)
plt.figure(figsize=(16, 6))
for i, k in enumerate(["4", "8", "16"]):
    plt.bar(x_indexes + i * bar_width, time_dict[k], width=bar_width, label=f"{k} Layers")
plt.xlabel("Image Index")
plt.ylabel("Inference Time (s)")
plt.title("Inference Time per Image")
plt.xticks(x_indexes + bar_width, [str(i+1) for i in range(30)])
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(results_dir, "inference_time_per_image.png"))
plt.close()
print("✅ Saved inference time bar graph per image.")


FileNotFoundError: [Errno 2] No such file or directory: '4_layers_model/best_model_4_4.pth'

In [1]:
# === Average Inference Time Bar Graph (Model-wise) ===
avg_times = {k: np.mean(time_dict[k]) for k in ["4", "8", "16"]}

models_list = ["4 Layers", "8 Layers", "16 Layers"]
avg_time_values = [avg_times["4"], avg_times["8"], avg_times["16"]]

plt.figure(figsize=(8, 6))
bars = plt.bar(models_list, avg_time_values, color=['skyblue', 'lightgreen', 'salmon'])
plt.xlabel("Model")
plt.ylabel("Average Inference Time (s)")
plt.title("Average Inference Time per Model")
plt.grid(axis='y')

# Add the average time value on top of each bar
for bar, avg_time in zip(bars, avg_time_values):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2.0, yval + 0.0005, f"{avg_time:.4f}s", ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig(os.path.join(results_dir, "average_inference_time_per_model.png"))
plt.close()
print("✅ Saved average inference time bar graph per model.")


NameError: name 'np' is not defined